In [1]:
import os
import time
import tensorflow as tf
from tensorflow.keras import models, layers, Model, callbacks
from tensorflow.keras.metrics import AUC, Precision, Recall
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from process_data import process_data
from nn_gridsearch import logdir

In [2]:
data = process_data()

In [3]:
for k, v in data.items():
    print(k)
    print(type(v))

x_train
<class 'numpy.ndarray'>
x_train_processed
<class 'numpy.ndarray'>
x_test
<class 'numpy.ndarray'>
x_test_processed
<class 'numpy.ndarray'>
y_train
<class 'numpy.ndarray'>
y_test
<class 'numpy.ndarray'>
pipeline
<class 'sklearn.pipeline.Pipeline'>


In [4]:
x_train_processed = data['x_train_processed']
y_train = data['y_train']

In [5]:
x_train_processed.shape

(1047, 11)

> note: old model constrution, using Sequential API

In [6]:
#model = models.Sequential()
#model.add(layers.Dense(32, activation='relu', input_shape=(x_train_processed.shape[1],)))
#model.add(layers.Dense(32, activation='relu'))
#model.add(layers.Dense(1, activation='sigmoid'))


> changed to functional API --> more flexibility, for future expansion

In [7]:
input_ = layers.Input(shape=x_train_processed.shape[1:])
hidden_0 = layers.Dense(80, activation='relu')(input_)
hidden_1 = layers.Dense(80, activation='relu')(hidden_0)
hidden_2 = layers.Dense(80, activation='relu')(hidden_1)
hidden_3 = layers.Dense(80, activation='relu')(hidden_2)
hidden_4 = layers.Dense(80, activation='relu')(hidden_3)
hidden_5 = layers.Dense(80, activation='relu')(hidden_4)
output = layers.Dense(1, activation='sigmoid')(hidden_5)
model_custom = Model(inputs=[input_], outputs=[output])

In [8]:
#model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model_custom.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', Precision(), Recall(), AUC()])

In [9]:
model_custom.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 11)]              0         
_________________________________________________________________
dense (Dense)                (None, 80)                960       
_________________________________________________________________
dense_1 (Dense)              (None, 80)                6480      
_________________________________________________________________
dense_2 (Dense)              (None, 80)                6480      
_________________________________________________________________
dense_3 (Dense)              (None, 80)                6480      
_________________________________________________________________
dense_4 (Dense)              (None, 80)                6480      
_________________________________________________________________
dense_5 (Dense)              (None, 80)                6480  

In [10]:
history = model_custom.fit(
    data['x_train_processed'], data['y_train'], validation_split=.2, epochs=300,
    callbacks=[
        EarlyStopping(patience=10, monitor='val_loss', mode='min'),
        TensorBoard(logdir('custom_model'))])

Epoch 1/300
27/27 [==============================] - 2s 39ms/step - loss: 0.4905 - accuracy: 0.7885 - precision: 0.7793 - recall: 0.5608 - auc: 0.8266 - val_loss: 0.6320 - val_accuracy: 0.7714 - val_precision: 0.8800 - val_recall: 0.5116 - val_auc: 0.8167
Epoch 2/300
27/27 [==============================] - 1s 21ms/step - loss: 0.4313 - accuracy: 0.8172 - precision: 0.7678 - recall: 0.6926 - auc: 0.8508 - val_loss: 0.7047 - val_accuracy: 0.6952 - val_precision: 1.0000 - val_recall: 0.2558 - val_auc: 0.8084
Epoch 3/300
27/27 [==============================] - 0s 18ms/step - loss: 0.4270 - accuracy: 0.8292 - precision: 0.8341 - recall: 0.6453 - auc: 0.8525 - val_loss: 0.5290 - val_accuracy: 0.7762 - val_precision: 0.8980 - val_recall: 0.5116 - val_auc: 0.8087
Epoch 4/300
27/27 [==============================] - 0s 13ms/step - loss: 0.4193 - accuracy: 0.8256 - precision: 0.8205 - recall: 0.6486 - auc: 0.8598 - val_loss: 0.4964 - val_accuracy: 0.7905 - val_precision: 0.7917 - val_recall: 0

In [11]:
%load_ext tensorboard
%tensorboard --logdir ./custom_logs

In [44]:
model_gridsearch = models.load_model('./models/titanic_gridsearch_2021_10_10_02_28.h5')

In [42]:
model_gridsearch.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hidden_layer_0_relu_alpha_0. (None, 39)                468       
_________________________________________________________________
batch_normalization_8 (Batch (None, 39)                156       
_________________________________________________________________
activation_8 (Activation)    (None, 39)                0         
_________________________________________________________________
dropout_0_20 (Dropout)       (None, 39)                0         
_________________________________________________________________
hidden_layer_1_relu_alpha_0. (None, 39)                1560      
_________________________________________________________________
batch_normalization_9 (Batch (None, 39)                156       
_________________________________________________________________
activation_9 (Activation)    (None, 39)               

In [39]:
model_custom.evaluate(data['x_test_processed'], data['y_test'])

9/9 [==============================] - 0s 13ms/step - loss: 0.5507 - accuracy: 0.7595 - precision: 0.8395 - recall: 0.5763 - auc: 0.8535


[0.550728440284729,
 0.7595419883728027,
 0.8395061492919922,
 0.5762711763381958,
 0.853489875793457]

In [45]:
model_gridsearch.evaluate(data['x_test_processed'], data['y_test'])

9/9 [==============================] - 1s 12ms/step - loss: 0.5056 - accuracy: 0.7710 - precision_1: 0.8816 - recall_1: 0.5678 - auc_1: 0.8524


[0.5055856704711914,
 0.7709923386573792,
 0.8815789222717285,
 0.5677965879440308,
 0.8524011373519897]

> todo next - function to build model, sklearn wrapper from keras, pipelines, random grid search for model!